# C13 project : rating prediction

Author: Ignasi

    0.1 Import package
    0.2 Describe data base 
    1.0 Feature engineering
        * Drop columns
        * transform categorical data
        * Merge data by canal
        * 
     

### 0.1 import package

In [1]:
#import package
#import pandas as pd
#import numpy as np

import warnings
warnings.filterwarnings('ignore')

# Load modules.
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

from sklearn import linear_model, metrics, model_selection
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from statsmodels.tools.eval_measures import rmse

import itertools

In [2]:
df = pd.read_csv('D:\datacanal13\CSVmodelo\entradaModelo.csv', sep=',', error_bad_lines=False,header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,2018-01-01,13:00,1,2018,1,0,1,1,6,45,...,0.040150,0.025769,0.045945,0.022039,0.038369,0.023148,0.082364,0.043027,0.012170,0.033241
1,2018-01-01,13:15,1,2018,1,0,1,1,6,45,...,0.041058,0.035029,0.050092,0.032723,0.044275,0.028089,0.105945,0.054492,0.005684,0.000000
2,2018-01-01,13:30,1,2018,1,0,1,1,6,45,...,0.084393,0.065127,0.108027,0.057281,0.093691,0.053283,0.134639,0.065339,0.033241,0.000000
3,2018-01-01,13:45,1,2018,1,0,1,1,6,45,...,0.108196,0.079961,0.120651,0.065998,0.118428,0.060838,0.139627,0.065078,0.000000,0.000000
4,2018-01-01,14:00,1,2018,1,0,1,1,6,45,...,0.054143,0.039312,0.064884,0.041394,0.068947,0.044276,0.136907,0.063837,0.000000,0.006662


In [4]:
#add the name of the columns
df.columns=['start_date','HORA','CANAL','AÑO','MES','DIA','FERIADO','SEMANA_MES','HORA_INICIO',
            'MIN_INICIO','HORA_FIN','MIN_FIN','COD_UNIVERSAL','PROGRAMA',
            'BLOQUES','TANDAS','DURACION_TOTAL','DURACION_CONTENIDO',"Extra",
           'GENERO','SUBGENERO','TEMPORADA','CAPITULO','ROSTROS', 'ROSTROS_INV',
           'INDIVIDUOHMABC25_64_ACTUAL','INDIVIDUOHMABC25_64_AVG4',
           'INDIVIDUOHMABC25_64_DST4','INDIVIDUOHMABC25_64_AVG8',
           'INDIVIDUOHMABC25_64_DST8','INDIVIDUOHMABC25_64_AVG12',
           'INDIVIDUOHMABC25_64_DST12','INDIVIDUOHMABC25_64_AVGALL',
           'INDIVIDUOHMABC25_64_DSTALL','INDIVIDUOHMABC25_64_ANTERIOR',
           'INDIVIDUOHMABC25_64_SIGUIENTE']

In [5]:
#df[(df.BLOQUES==28)&(df.CANAL==5)]

In [6]:
#aux_df=df[['CANAL','AÑO','MES','SEMANA_MES','DIA','BLOQUES','PROGRAMA','INDIVIDUOHMABC25_64_ACTUAL']].sort_values(by=['CANAL','AÑO','MES','SEMANA_MES','DIA','BLOQUES'], ascending=True)

There is a column that is not clear about the information it stores, until now it will be called **"Extra"**

### 0.2 Describe data base

In [7]:
#Describe data base
#df.describe().transpose()

In [8]:
#len(df.CANAL.unique()), len(df.COD_UNIVERSAL.unique()),len(df.BLOQUES.unique()),len(df.PROGRAMA.unique())

Relevant information from the database:
1.  Number of tv channels: 4 
    * 1: CHV
    * 3: MEGA
    * 4: TVN
    * 5: C13
2.  Number of universal code: 1174
3.  Number of programs: 1115
4.  Number of time block: 96

The universal code is supposed to be unique, and it should have the same number of programs. In this case this does not happen.

In the database there are differences of observations per channel, this is because the start and end vary depending on the day and the channel.


In [9]:
for canal in df.CANAL.unique():
    print(df[df.CANAL==canal].shape)


(79725, 36)
(79402, 36)
(81972, 36)
(81919, 36)


## 1.0 Feature engineering

* Drop columns
    * remove columns with missing data
    * remove columns anterior y siguiente, because they are data that in practice you will never know
* Transform categorical data
    * BLOQUES
* Separate database by channel
    * Transform categorical data:  ... This is done later so as not to generate so many columns
        * PROGRAMA
    * Transform to dummy columns
        * PROGRAMA
* Merge data by canal:
    * Type of merge to be performed: Left
* Transform to dummy columns
    * BLOQUES

In [10]:
#Drop columns
df.drop(columns=["Extra",'COD_UNIVERSAL','TEMPORADA','CAPITULO','ROSTROS', 'ROSTROS_INV','INDIVIDUOHMABC25_64_ANTERIOR','INDIVIDUOHMABC25_64_SIGUIENTE' ], inplace=True)

In [11]:
# Transform categorical data
to_covert_cat = ['BLOQUES']
for col in to_covert_cat:
    df[col] = df[col].astype('category')

In [12]:
#Separate database by channel
lista_canales = df.CANAL.unique()
merge_list=['AÑO','MES','DIA','FERIADO','SEMANA_MES', 'BLOQUES']
to_covert_cat_1 = ['GENERO','SUBGENERO']
use_programa=True
list_df_canales=[]
for canal in lista_canales:
    aux=df[df.CANAL == canal].copy()
    if canal==5:
        list_programa=[]
        for lista in ['AÑO','MES','SEMANA_MES','DIA','BLOQUES','PROGRAMA','start_date','HORA']:
        
            list_programa.append(aux[lista].tolist())
        
    aux.drop(columns=['CANAL','PROGRAMA','start_date','HORA'], inplace=True)
    if use_programa:
        aux= pd.get_dummies(aux, columns=to_covert_cat_1)
    
    cols = aux.columns[~aux.columns.isin(merge_list)]
    aux.rename(columns = dict(zip(cols, cols + '_'+str(canal))), inplace=True)
    list_df_canales.append([aux,canal])

    
#Merge data by canal
df_final=list_df_canales[3][0]
for df_canal in list_df_canales[:-1]:
    #print(df_canal[0].shape)
    df_final=df_final.merge(df_canal[0], on=merge_list,how='left')


In [13]:
see_na=False
if see_na:
    list_columns_na=df_final.columns[df_final.isna().any()].tolist()
    print(df_final[list_columns_na].isna().sum())
    

In [14]:
#Transform to dummy columns
df_final= pd.get_dummies(df_final, columns=to_covert_cat)
df_final=df_final.sort_values(by=['AÑO','MES','SEMANA_MES','DIA'], ascending=True)
df_final=df_final.reset_index(drop=True)
df_final.fillna(0,inplace=True)

In [15]:
lista_columns = ['INDIVIDUOHMABC25_64_ACTUAL_5']

In [16]:
df_X = df_final[df_final.columns.difference(lista_columns)].copy()
df_Y = df_final[lista_columns].copy()

## Split data


In [17]:
list_tuple_week=list(set((i,j,k) for i,j,k in zip(df_final.AÑO, df_final.MES,df_final.SEMANA_MES)))
list_tuple_week.sort()

In [19]:
list_index=[]
for tuplas in list_tuple_week:
    list_index.append(df_final[(df_final.AÑO==tuplas[0])&(df_final.MES==tuplas[1])&(df_final.SEMANA_MES==tuplas[2])].index[-1])

In [20]:
year_train=2018
index_year_train=df_final[(df_final.AÑO==year_train)].index[-1]

In [3]:
for t in range(1,7):
    # Run sliding window regression models

    # Define test data
    X_test = df_X.iloc[index_year_train+1:,:]
    Y_test = df_Y.iloc[index_year_train+1:,:]
    Yhat_test = np.zeros_like(Y_test)
    Yhat_test_all = np.zeros((Yhat_test.shape[0],2))
    print(t)

1
2
3
4
5
6


In [21]:
#list_index.index(index_year_train)

In [19]:
# Run sliding window regression models

# Define test data
X_test = df_X.iloc[index_year_train+1:,:]
Y_test = df_Y.iloc[index_year_train+1:,:]
Yhat_test = np.zeros_like(Y_test)
Yhat_test_all = np.zeros((Yhat_test.shape[0],2))
                         
# Iterate list of index
for i in range(list_index.index(index_year_train),len(list_index)-1):
#for i in range(list_index.index(index_year_train),67):
    print("Training for test point:",list_tuple_week[i] )
    print("Test point:",list_tuple_week[i+1] )
    # Set training window
    lo = list_index[i]+1
    hi = list_index[i+1]+1
    X_train_i = df_X.iloc[:lo,:]
    Y_train_i = df_Y.iloc[:lo,:]
    X_test_i = df_X.iloc[lo:hi,:]
    
    # OLS
    reg = linear_model.LinearRegression(normalize=False).fit(X_train_i, Y_train_i)
    
    # Predict next values
    #Yhat_test_all[0,N*i:N*(i+1)] = reg.predict(X_test_i)
    Yhat_test_i = reg.predict(X_test_i)
    Yhat_test[lo-index_year_train-1:hi-index_year_train-1,:] = Yhat_test_i
    Yhat_test_all[lo-index_year_train-1:hi-index_year_train-1,0] = reg.predict(X_test_i).reshape((-1))
#     Yhat_test_all[1,N*i:N*(i+1),:] = reg.predict(X_test_i)
    print("OLS")
    # WLS
#     weights = np.tile(dem[:,0], train_win)
#     reg = linear_model.LinearRegression(normalize=True).fit(X_train_i, Y_train_i, sample_weight=weights)
    
# #     # LASSO
    #num_train = int(X_train_i.shape[0]/N)
    #lasso = linear_model.MultiTaskElasticNetCV(l1_ratio=1,normalize=True).fit(X_train_i, Y_train_i)
    #reg = MultiOutputRegressor(linear_model.Lasso(warm_start=True, alpha=lasso.alpha_, normalize=True)).fit(X_train_i, Y_train_i)
    #Yhat_test_all[1,lo-index_year_train-1:hi-index_year_train-1] = reg.predict(X_test_i).reshape((-1))
    
# #     # Ridge
#     num_train = int(X_train_i.shape[0]/N)
    #net = linear_model.MultiTaskElasticNetCV(l1_ratio=1e-8,normalize=True).fit(X_train_i, Y_train_i)
    #reg = MultiOutputRegressor(linear_model.Ridge(alpha=net.alpha_, normalize=True)).fit(X_train_i, Y_train_i)
    #Yhat_test_all[2,lo-index_year_train-1:hi-index_year_train-1]= reg.predict(X_test_i).reshape((-1))
# #     # XGBoost
    reg = MultiOutputRegressor(XGBRegressor()).fit(X_train_i, Y_train_i)
    Yhat_test_all[lo-index_year_train-1:hi-index_year_train-1,1]= reg.predict(X_test_i).reshape((-1))
    print("XGBoost")
# #     # MLP
#     reg = MLPRegressor().fit(X_train_i, Y_train_i)
#     Yhat_test_all[5,N*i:N*(i+1),:] = reg.predict(X_test_i)
    print("Index pred:{}".format((lo-index_year_train-1,hi-index_year_train-1)))
    
    

Training for test point: (2018, 12, 6)
Test point: (2019, 1, 1)
OLS
XGBoost
Index pred:(0, 465)
Training for test point: (2019, 1, 1)
Test point: (2019, 1, 2)
OLS
XGBoost
Index pred:(465, 1033)
Training for test point: (2019, 1, 2)
Test point: (2019, 1, 3)
OLS
XGBoost
Index pred:(1033, 1603)
Training for test point: (2019, 1, 3)
Test point: (2019, 1, 4)
OLS
XGBoost
Index pred:(1603, 2170)
Training for test point: (2019, 1, 4)
Test point: (2019, 1, 5)
OLS
XGBoost
Index pred:(2170, 2489)
Training for test point: (2019, 1, 5)
Test point: (2019, 2, 1)
OLS
XGBoost
Index pred:(2489, 2707)
Training for test point: (2019, 2, 1)
Test point: (2019, 2, 2)
OLS
XGBoost
Index pred:(2707, 3247)
Training for test point: (2019, 2, 2)
Test point: (2019, 2, 3)
OLS
XGBoost
Index pred:(3247, 3792)
Training for test point: (2019, 2, 3)
Test point: (2019, 2, 4)
OLS
XGBoost
Index pred:(3792, 4363)
Training for test point: (2019, 2, 4)
Test point: (2019, 2, 5)
OLS
XGBoost
Index pred:(4363, 4699)
Training for t

OLS
XGBoost
Index pred:(38159, 38694)
Training for test point: (2020, 4, 4)
Test point: (2020, 4, 5)
OLS
XGBoost
Index pred:(38694, 39003)
Training for test point: (2020, 4, 5)
Test point: (2020, 5, 1)
OLS
XGBoost
Index pred:(39003, 39226)
Training for test point: (2020, 5, 1)
Test point: (2020, 5, 2)
OLS
XGBoost
Index pred:(39226, 39762)
Training for test point: (2020, 5, 2)
Test point: (2020, 5, 3)
OLS
XGBoost
Index pred:(39762, 40298)
Training for test point: (2020, 5, 3)
Test point: (2020, 5, 4)
OLS
XGBoost
Index pred:(40298, 40832)
Training for test point: (2020, 5, 4)
Test point: (2020, 5, 5)
OLS
XGBoost
Index pred:(40832, 41373)
Training for test point: (2020, 5, 5)
Test point: (2020, 6, 1)
OLS
XGBoost
Index pred:(41373, 41899)
Training for test point: (2020, 6, 1)
Test point: (2020, 6, 2)
OLS
XGBoost
Index pred:(41899, 42432)
Training for test point: (2020, 6, 2)
Test point: (2020, 6, 3)
OLS
XGBoost
Index pred:(42432, 42971)
Training for test point: (2020, 6, 3)
Test point: (20

In [56]:
new_columns=list_programa[index_year_train+1:]


[2019, 1, 1, 1, 35, 'CUBOX']

In [87]:
Y_test.loc[:,"OLS"]=Yhat_test_all[:,0]
Y_test.loc[:,"XGB"]=Yhat_test_all[:,1]

lista_aux=['AÑO','MES','SEMANA_MES','DIA','BLOQUES','PROGRAMA']

for i in range(len(lista_aux)):
    Y_test[lista_aux[i]]=list_programa[i][index_year_train+1:]

In [88]:
Y_test=Y_test[['AÑO','MES','SEMANA_MES','DIA','BLOQUES','PROGRAMA','INDIVIDUOHMABC25_64_ACTUAL_5','OLS',"XGB"]].copy()
Y_test['OLS'] = np.where(Y_test['OLS'] < 0, 0, Y_test['OLS'])
Y_test['XGB'] = np.where(Y_test['XGB'] < 0, 0, Y_test['XGB'])

In [89]:
Y_test#.reindex(columns=[3,0,1,2])

,AÑO,MES,SEMANA_MES,DIA,BLOQUES,PROGRAMA,INDIVIDUOHMABC25_64_ACTUAL_5,OLS,XGB
28726,2019,1,1,1,34,CUBOX,0.041340,0.000000,0.144602
28727,2019,1,1,1,35,CUBOX,0.029887,0.000000,0.127133
28728,2019,1,1,1,36,CUBOX,0.051552,0.000000,0.111829
28729,2019,1,1,1,37,CUBOX,0.010832,0.000000,0.110683
28730,2019,1,1,1,38,CUBOX,0.030159,0.000000,0.128900
...,...,...,...,...,...,...,...,...,...
81914,2020,10,5,4,105,SUPER SERIES,0.163070,0.104704,0.117785
81915,2020,10,5,4,106,SUPER SERIES,0.155364,0.137962,0.125717
81916,2020,10,5,4,107,SUPER SERIES,0.143545,0.142975,0.136993
81917,2020,10,5,4,108,SUPER SERIES,0.169082,0.131612,0.108461


In [90]:
Y_test.to_csv("model_predict_v2.csv")

In [ ]:
# Run sliding window regression models

# Define test data
X_test = df_X.iloc[index_year_train+1:,:]
Y_test = df_Y.iloc[index_year_train+1:,:]
Yhat_test = np.zeros_like(Y_test)
Yhat_test_all = np.zeros((Yhat_test.shape[0],4))
                         
# Iterate list of index
for i in range(list_index.index(index_year_train),len(list_index)-1):
#for i in range(list_index.index(index_year_train),67):
    print("Training for test point:",list_tuple_week[i] )
    print("Test point:",list_tuple_week[i+1] )
    # Set training window
    lo = list_index[i]+1
    hi = list_index[i+1]+1
    X_train_i = df_X.iloc[:lo,:]
    Y_train_i = df_Y.iloc[:lo,:]
    X_test_i = df_X.iloc[lo:hi,:]
    
    
# #     # Elastic net
#   num_train = int(X_train_i.shape[0]/N)
    net = linear_model.MultiTaskElasticNetCV(l1_ratio=0.5,normalize=True).fit(X_train_i, Y_train_i)
    reg = MultiOutputRegressor(linear_model.ElasticNet(alpha=net.alpha_, normalize=True)).fit(X_train_i, Y_train_i)
    Yhat_test_all[lo-index_year_train-1:hi-index_year_train-1,2] = reg.predict(X_test_i).reshape((-1))
    print('elastic-net')
    
    #Yhat_test_all[2,lo-index_year_train-1:hi-index_year_train-1]= reg.predict(X_test_i).reshape((-1))
# #     # XGBoost
    #reg = MultiOutputRegressor(XGBRegressor()).fit(X_train_i, Y_train_i)
    #Yhat_test_all[lo-index_year_train-1:hi-index_year_train-1,1]= reg.predict(X_test_i).reshape((-1))
    #print("XGBoost")
# #     # MLP
#     reg = MLPRegressor().fit(X_train_i, Y_train_i)
#     Yhat_test_all[5,N*i:N*(i+1),:] = reg.predict(X_test_i)
    print("Index pred:{}".format((lo-index_year_train-1,hi-index_year_train-1)))
    

Training for test point: (2018, 12, 6)
Test point: (2019, 1, 1)
elastic-net
Index pred:(0, 465)
Training for test point: (2019, 1, 1)
Test point: (2019, 1, 2)
elastic-net
Index pred:(465, 1033)
Training for test point: (2019, 1, 2)
Test point: (2019, 1, 3)
elastic-net
Index pred:(1033, 1603)
Training for test point: (2019, 1, 3)
Test point: (2019, 1, 4)
elastic-net
Index pred:(1603, 2170)
Training for test point: (2019, 1, 4)
Test point: (2019, 1, 5)
elastic-net
Index pred:(2170, 2489)
Training for test point: (2019, 1, 5)
Test point: (2019, 2, 1)
elastic-net
Index pred:(2489, 2707)
Training for test point: (2019, 2, 1)
Test point: (2019, 2, 2)
elastic-net
Index pred:(2707, 3247)
Training for test point: (2019, 2, 2)
Test point: (2019, 2, 3)
elastic-net
Index pred:(3247, 3792)
Training for test point: (2019, 2, 3)
Test point: (2019, 2, 4)
elastic-net
Index pred:(3792, 4363)
Training for test point: (2019, 2, 4)
Test point: (2019, 2, 5)
elastic-net
Index pred:(4363, 4699)
Training for t

In [ ]:
Y_test.loc[:,"NET"]=Yhat_test_all[:,2]

lista_aux=['AÑO','MES','SEMANA_MES','DIA','BLOQUES','PROGRAMA','start_date','HORA']

for i in range(len(lista_aux)):
    Y_test[lista_aux[i]]=list_programa[i][index_year_train+1:]

In [ ]:
Y_test.to_csv("model_predict_net_v2.csv")

In [4]:
import itertools

In [45]:
# Run sliding window regression models

# Define test data
X_test = df_X.iloc[index_year_train+1:,:]
Y_test = df_Y.iloc[index_year_train+1:,:]
Yhat_test = np.zeros_like(Y_test)
Yhat_test_all_XGB = np.zeros((Yhat_test.shape[0],16))
list_errores=[]
                         
# Iterate list of index
for i in range(list_index.index(index_year_train),len(list_index)-1):
#for i in range(list_index.index(index_year_train),67):
    print("Training for test point:",list_tuple_week[i] )
    print("Test point:",list_tuple_week[i+1] )
    # Set training window
    lo = list_index[i]+1
    hi = list_index[i+1]+1
    X_train_i = df_X.iloc[:lo,:]
    Y_train_i = df_Y.iloc[:lo,:]
    X_test_i = df_X.iloc[lo:hi,:]
    Y_test_i = df_Y.iloc[lo:hi,:]
    lista_errores_semana=[]
    
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        #'min_child_weight': [1, 3, 5],
        'n_estimators' : [100, 200],
    }
    
    keys, values = zip(*param_tuning.items())
    permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]
    
    for i in range(len(permutations_dicts)):
        reg = MultiOutputRegressor(XGBRegressor(**permutations_dicts[i])).fit(X_train_i, Y_train_i)
        lista_errores_semana.append(((Y_train_i - reg.predict(X_train_i)) ** 2).mean().values)
        lista_errores_semana.append(((Y_test_i - reg.predict(X_test_i)) ** 2).mean().values)
        
        
        Yhat_test_all_XGB[lo-index_year_train-1:hi-index_year_train-1,i]= reg.predict(X_test_i).reshape((-1))
    
    
    list_errores.append(lista_errores_semana)
    print("XGBoost")

    print("Index pred:{}".format((lo-index_year_train-1,hi-index_year_train-1)))

Training for test point: (2018, 12, 6)
Test point: (2019, 1, 1)
XGBoost
Index pred:(0, 465)
Training for test point: (2019, 1, 1)
Test point: (2019, 1, 2)
XGBoost
Index pred:(465, 1033)
Training for test point: (2019, 1, 2)
Test point: (2019, 1, 3)
XGBoost
Index pred:(1033, 1603)
Training for test point: (2019, 1, 3)
Test point: (2019, 1, 4)
XGBoost
Index pred:(1603, 2170)
Training for test point: (2019, 1, 4)
Test point: (2019, 1, 5)
XGBoost
Index pred:(2170, 2489)
Training for test point: (2019, 1, 5)
Test point: (2019, 2, 1)
XGBoost
Index pred:(2489, 2707)
Training for test point: (2019, 2, 1)
Test point: (2019, 2, 2)
XGBoost
Index pred:(2707, 3247)
Training for test point: (2019, 2, 2)
Test point: (2019, 2, 3)
XGBoost
Index pred:(3247, 3792)
Training for test point: (2019, 2, 3)
Test point: (2019, 2, 4)
XGBoost
Index pred:(3792, 4363)
Training for test point: (2019, 2, 4)
Test point: (2019, 2, 5)
XGBoost
Index pred:(4363, 4699)
Training for test point: (2019, 2, 5)
Test point: (201

XGBoost
Index pred:(39226, 39762)
Training for test point: (2020, 5, 2)
Test point: (2020, 5, 3)
XGBoost
Index pred:(39762, 40298)
Training for test point: (2020, 5, 3)
Test point: (2020, 5, 4)
XGBoost
Index pred:(40298, 40832)
Training for test point: (2020, 5, 4)
Test point: (2020, 5, 5)
XGBoost
Index pred:(40832, 41373)
Training for test point: (2020, 5, 5)
Test point: (2020, 6, 1)
XGBoost
Index pred:(41373, 41899)
Training for test point: (2020, 6, 1)
Test point: (2020, 6, 2)
XGBoost
Index pred:(41899, 42432)
Training for test point: (2020, 6, 2)
Test point: (2020, 6, 3)
XGBoost
Index pred:(42432, 42971)
Training for test point: (2020, 6, 3)
Test point: (2020, 6, 4)
XGBoost
Index pred:(42971, 43505)
Training for test point: (2020, 6, 4)
Test point: (2020, 6, 5)
XGBoost
Index pred:(43505, 43652)
Training for test point: (2020, 6, 5)
Test point: (2020, 7, 1)
XGBoost
Index pred:(43652, 44024)
Training for test point: (2020, 7, 1)
Test point: (2020, 7, 2)
XGBoost
Index pred:(44024, 445

In [47]:
for i in range(16):
    Y_test.loc[:,"XGB_"+str(i)]=Yhat_test_all_XGB[:,i]

lista_aux=['AÑO','MES','SEMANA_MES','DIA','BLOQUES','PROGRAMA','start_date','HORA']

for i in range(len(lista_aux)):
    Y_test[lista_aux[i]]=list_programa[i][index_year_train+1:]

In [49]:
Y_test.to_csv("model_predict_xgb_tunning_v2.csv")

In [58]:
error_xgb_tunning=pd.DataFrame.from_records(list_errores).apply(lambda x: x.explode() if x.name in list(range(32)) else x)

In [59]:
error_xgb_tunning["week"]=list_tuple_week[63:]

In [60]:
error_xgb_tunning.to_csv("error_predict_xgb_tunning_v2.csv")

In [61]:
error_xgb_tunning

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,week
0,0.019239,0.018985,0.005727,0.005336,0.018640,0.018656,0.005014,0.005086,0.018156,0.018144,...,0.002986,0.001370,0.003160,0.000943,0.003172,0.000520,0.003249,0.000218,0.003263,"(2019, 1, 1)"
1,0.019238,0.018465,0.005732,0.004964,0.018654,0.017994,0.005025,0.004421,0.018174,0.017420,...,0.002795,0.001350,0.002798,0.000957,0.002784,0.000535,0.002730,0.000239,0.002773,"(2019, 1, 2)"
2,0.019237,0.024949,0.005728,0.007698,0.018656,0.025370,0.005028,0.007773,0.018182,0.024884,...,0.003277,0.001393,0.003303,0.000937,0.003263,0.000551,0.004037,0.000250,0.004007,"(2019, 1, 3)"
3,0.019279,0.021439,0.005723,0.005838,0.018702,0.021216,0.005039,0.005645,0.018217,0.020507,...,0.002754,0.001315,0.002778,0.000937,0.002807,0.000541,0.002862,0.000259,0.002860,"(2019, 1, 4)"
4,0.019293,0.025225,0.005716,0.007062,0.018722,0.024532,0.005039,0.006341,0.018235,0.023794,...,0.002051,0.001358,0.002224,0.000961,0.002311,0.000528,0.002220,0.000240,0.002211,"(2019, 1, 5)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,0.019722,0.015309,0.005855,0.003083,0.019033,0.014742,0.005209,0.002780,0.018654,0.014691,...,0.002027,0.001746,0.001933,0.001392,0.001943,0.000941,0.001952,0.000566,0.002023,"(2020, 10, 1)"
110,0.019715,0.018417,0.005850,0.004013,0.019025,0.017939,0.005203,0.003773,0.018648,0.017907,...,0.001670,0.001760,0.001691,0.001396,0.001743,0.000908,0.001649,0.000575,0.001686,"(2020, 10, 2)"
111,0.019706,0.021225,0.005837,0.005413,0.019020,0.020861,0.005196,0.005255,0.018645,0.021029,...,0.001999,0.001755,0.002076,0.001385,0.002077,0.000920,0.002204,0.000571,0.002246,"(2020, 10, 3)"
112,0.019706,0.022053,0.005828,0.006108,0.019028,0.021682,0.005193,0.006137,0.018653,0.022304,...,0.003263,0.001745,0.003660,0.001388,0.003766,0.000922,0.004327,0.000580,0.004417,"(2020, 10, 4)"


# Form results dataframe
predic = np.zeros((, N, ))
actual = np.zeros((, N, ))
for t in range():
    predic[t,:,:] = Yhat_test[,:]
    actual[t,:,:] = Y_test[,:]